Создание чат-бота-"барахольщика": по продуктовому запрос он будет рекомендовать товары, по остальным запросам он бует отвечать "болталкой" без фолбека.

# Task

* Обучить классификатор: продуктовый запрос vs. всё остальное (продуктовым можно считать запрос, который равен названию или описанию товара).
* добавить логику поиска похожих товаров по продуктовому запросу.
* Вся логика должна быть завёрнута в метод get_answer(). Ответ на продуктовый запрос должен иметь вид "product_id title".

Автотесты применяются к методу get_answer и проверяют несколько базовых сценариев:

 
 - assert(get_answer(“Юбка детская ORBY”).startswith(“58e3cfe6132ca50e053f5f82”))
 - assert(not get_answer(“Где ключи от танка”).startswith(“5”)) 


КРИТЕРИИ ПРОВЕРКИ:

* Обучен классификатор «товарный запрос vs. болталка»:

* Осуществлён препроцессинг текста (как минимум удаление знаков препинания, приведение к нижнему регистру, стемминг/лемматизация).
* Текст векторизирован любым из изученных способов (CountVectorizer, TfidfVectorizer, HashingVectorizer, Word2Vec).
* Выборка разделена на обучающую и валидационную.
* Обучен классификатор с расчётом метрик на валидации (любое семейство алгоритмов, но предпочтительнее просто логистическая регрессия).
* Модель сохранена и при применении загружается из pkl-файла (или аналога).


→ Реализован поиск похожих товаров в контентной части бота

* Все названия товаров свёрнуты в векторное представление Word2Vec (предобученном или обученном на исходном датасете).
* Построен индекс по названиям документов.
* Для товарных запросов реализован поиск в индексе (запрос также оборачивается Word2Vec, происходит проход в индекс).

→ Реализована болталка

* Все вопросы из датасета свёрнуты Word2Vec в векторное представление.
* Построен индекс по вопросам.
* На запрос в болталку происходит поиск ближайшего вопроса и возвращается ответ на этот вопрос.

# Imports

In [1]:
# data  
import pandas as pd 
import numpy as np

# text 
import string 
import pymorphy2
import codecs
from stop_words import get_stop_words
from gensim.models import Word2Vec


# models
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection  

# serialization 
import pickle 

# Index  
import annoy

# Functions

In [2]:
# Очистка от пунктуации 
punctuation = set(string.punctuation)

# стоп слова
stop_words = set(get_stop_words('ru'))

# лемматизация 
morpher = pymorphy2.MorphAnalyzer()

def preprocess_text(line):
    '''Text cleaning from punctuation, 
    register unification, returning normal form, excluding stop words'''
    item = str(line)
    item = ''.join(i for i in line.strip() if i not in punctuation).split()
    item = [morpher.parse(i.lower())[0].normal_form for i in item]
    item = [i for i in item if i not in stop_words and i != ""]
    item = ' '.join(item)
    return item


In [3]:
def get_answer(question):
    """Function preprocesses question text, composing a vector of question 
    as a sum of vectors for each word in question, 
    then finds out a nearest vector of reply and returns it's word representation"""
    
    preprocessed_question = preprocess_text(question) 
    
    with open('classifier.pkl', 'rb') as pkl_file: 
        classifier = pickle.load(pkl_file) # loading 
        vec = ngrams_vectorizer.transform([preprocessed_question])
        label = int(classifier.predict(vec))
    
    n_w2v = 0
    vector = np.zeros(100)
    
    if label == 1: # проход в продуктовый индекс
        for word in preprocessed_question:
            if word in model_w2v_prod.wv:
                vector += model_w2v_prod.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        answer_index = index_prod.get_nns_by_vector(vector, 1)
        return index_map_prod[answer_index[0]]
    
    else: # проход в индекс болталки
        for word in preprocessed_question:
            if word in chat_model.wv:
                vector += chat_model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        answer_index = index.get_nns_by_vector(vector, 1)
        return index_map[answer_index[0]]

In [5]:
def model_accuracy(classifier, feature_vec_train, label, feature_vec_val):
    """Function fits a model on feature vectors 
    given and returns accuracy score"""
    classifier.fit(feature_vec_train, label)
    predictions = classifier.predict(feature_vec_val)     
    return metrics.accuracy_score(predictions, y_val)

# EDA

### Reading

In [6]:
data_product = pd.read_csv('ProductsDataset.csv')
data_product

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...
...,...,...,...,...,...,...,...
35543,Юбка,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472,9.0,904,"{'zhenskaya_odezhda_platya_yubki_tip': 'Юбки',...",http://cache3.youla.io/files/images/360_360/5b...
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0,9.0,908,"{'brand_zhenskii': 'Chanel', 'zhenskaya_odezhd...",http://cache3.youla.io/files/images/360_360/5b...
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2,9.0,903,"{'zhenskaya_odezhda_razmer': '48-50 (XL)', 'zh...",http://cache3.youla.io/files/images/360_360/5b...
35546,Новая золотая ветровка,Женская ветровка размер 44-46. Цвет приглушённ...,5bd6c8fb2138bbc55745362c,9.0,903,"{'zhenskaya_odezhda_razmer': '44-46 (М)', 'zhe...",http://cache3.youla.io/files/images/360_360/5b...


In [7]:
data_product.isnull().sum()

title                0
descrirption      2011
product_id          12
category_id         12
subcategory_id      12
properties          12
image_links         15
dtype: int64

In [8]:
# Filling misses in id of products with "no id" 
data_product['product_id'] = data_product['product_id'].fillna('no id')

# Question classifying

Обучить классификатор: продуктовый запрос vs. всё остальное (продуктовым можно считать запрос, который равен названию или описанию товара).

Для классификатора запросов объединим данные: 
 - с названием товаров
 - и ответы на вопросы для чата. 

И добавим target - метку, 1 - для товаров, 0 - для вопросов чата


In [9]:
# Товарные данные 
data = {'issue': data_product.title, 'target': 1}
data = pd.DataFrame(data)

In [10]:
# данные из болталки для обучения 
sentences = []

# morpher = pymorphy2.MorphAnalyzer()
# sw = set(get_stop_words("ru"))
# exclude = set(string.punctuation)
c = 0

with codecs.open("Otvety.txt", "r", "utf-8") as fin:
    for line in fin:
        item = preprocess_text(line)
        sentences.append(item)
        c += 1
        if c > 35549:
            break

In [11]:
data_chat = {'issue': sentences, 'target': 0}
data_chat = pd.DataFrame(data_chat)

In [12]:
# сформируем фрейм для обучения классификатора, с сбалансированными классами 
df = pd.concat([data, data_chat], axis=0, ignore_index=True)

In [13]:
df.target.value_counts()

0    35550
1    35548
Name: target, dtype: int64

In [14]:
df[df.target == 0]

,issue,target
35548,,0
35549,,0
35550,вопрос тдв отдыхать лично советовать завести,0
35551,хомячок,0
35552,мужик йопарить собачка 50 кошка,0
...,...,...
71093,саров 10р,0
71094,череповец вологда 10 рубль,0
71095,автобус 10 руб траллейбус трамвай 6 руб маршру...,0
71096,50 деревяшекалматы,0


## data split

In [15]:
# splitting data 
x_train, x_val, y_train, y_val= model_selection.train_test_split(df['issue'], df['target'])

In [16]:
y_train = np.array(y_train)
y_val = np.array(y_val)

#### Tfidf vectorizing

In [17]:
# попробуем тфидф векторнуть и на этом обучить логрег
tfidf_vectorizer = TfidfVectorizer().fit(x_train.values) 
x_train_tfidf = tfidf_vectorizer.transform(x_train)
x_valid_tfidf = tfidf_vectorizer.transform(x_val)

#### Count vectorizing

In [18]:
count_vectorizer = CountVectorizer().fit(x_train.values)

In [19]:
x_train_count = count_vectorizer.transform(x_train)
x_valid_count = count_vectorizer.transform(x_val)

#### tfIdf n-grams

In [20]:
ngrams_vectorizer = TfidfVectorizer(ngram_range=(1, 3)).fit(x_train.values)

In [21]:
x_train_ngrams = ngrams_vectorizer.transform(x_train)
x_valid_ngrams = ngrams_vectorizer.transform(x_val)

In [22]:
x_train.values

array(['Зимний костюм Tokka Tribe 98 р-р', 'Ветровка летняя',
       'Интересное платье с цветным принтом,50 размер.', ...,
       'Замок игрушечный', 'Новое платье Next 2-6 мес.',
       'генералка хозяин временный ввоз кататься'], dtype=object)

## Classifier

In [23]:
clf = LogisticRegression()

#### Metrics 
Метрикой возьмем accuracy, так как классы по target сбалансированы.

In [24]:
# Linear Classifier на Count Vectors
accuracy = model_accuracy(clf, x_train_count, y_train, x_valid_count)
print("accuracy, Count Vectors: ", accuracy)

# Linear Classifier на Word Level TF IDF Vectors
accuracy = model_accuracy(clf, x_train_tfidf, y_train, x_valid_tfidf)
print("accuracy, WordLevel TF-IDF: ", accuracy)

# Linear Classifier на Ngram Level TF IDF Vectors
accuracy = model_accuracy(clf, x_train_ngrams, y_train, x_valid_ngrams)
print("accuracy, N-Gram Vectors: ", accuracy)

accuracy, Count Vectors:  0.9686638537271449
accuracy, WordLevel TF-IDF:  0.9688326300984529
accuracy, N-Gram Vectors:  0.9684388185654008


In [25]:
regressor = clf.fit(x_train_ngrams, y_train)

#### Classifier test

In [26]:
issue = 'юбка детская Orby'
vec = ngrams_vectorizer.transform([issue])
label = regressor.predict(vec)

#### Classifier saving

In [27]:
with open('classifier.pkl', 'wb') as output: 
    pickle.dump(regressor, output) # saving     

# Content
Реализован поиск похожих товаров в контентной части бота

 - Все названия товаров свёрнуты в векторное представление Word2Vec (предобученном или обученном на исходном датасете).
 - Построен индекс по названиям документов.
 - Для товарных запросов реализован поиск в индексе (запрос также оборачивается Word2Vec, происходит проход в индекс).

## Preprocessing text

Для векторизации нам нужно будет поместить целевую - id продукта - в контекст запроса. 

In [28]:
title_id = data_product['product_id'].astype(str) + '\t ' + data_product['title'].astype(str)
    
title_id 

0             58e3cfe6132ca50e053f5f82\t Юбка детская ORBY
1                     5667531b2b7f8d127d838c34\t Ботильоны
2                         59534826aaab284cba337e06\t Брюки
3          57de544096ad842e26de8027\t Продам детские шапки
4                        5ad4d2626c86cb168d212022\t Блузка
                               ...                        
35543                      5b5f181c62e1c6616a7f6472\t Юбка
35544     5bd6c8b29e94ba033d31f8d0\t Новый твидовый пиджак
35545     5bd6c8bc074b3e1c056f69b2\t Женская зимняя куртка
35546    5bd6c8fb2138bbc55745362c\t Новая золотая ветровка
35547                      5bd6c8fbaaab283b79142a1f\t Шарф
Length: 35548, dtype: object

In [29]:
title_id.apply(preprocess_text)

0             58e3cfe6132ca50e053f5f82 юбка детский orby
1                      5667531b2b7f8d127d838c34 ботильон
2                         59534826aaab284cba337e06 брюки
3         57de544096ad842e26de8027 продать детский шапка
4                        5ad4d2626c86cb168d212022 блузка
                              ...                       
35543                      5b5f181c62e1c6616a7f6472 юбка
35544     5bd6c8b29e94ba033d31f8d0 новый твидовый пиджак
35545     5bd6c8bc074b3e1c056f69b2 женский зимний куртка
35546    5bd6c8fb2138bbc55745362c новый золотой ветровка
35547                      5bd6c8fbaaab283b79142a1f шарф
Length: 35548, dtype: object

## Vectorizing, saving model

In [30]:
model_w2v_prod = Word2Vec(sentences=title_id, vector_size=100, min_count=1, window=5)
model_w2v_prod.save('model_w2v_prod')

## Index 
Строим индекс по продуктовым запросам

In [31]:
a = title_id[0].split('\t')# [-1]
a

['58e3cfe6132ca50e053f5f82', ' Юбка детская ORBY']

In [32]:
index_prod = annoy.AnnoyIndex(100, 'angular')

# строим словарь соответствия векторного представления буквенному
index_map_prod = {} 
counter = 0

for line in title_id:
    n_w2v = 0
    els = line.split('\t')
    index_map_prod[counter] = ' '. join(els)  # els
    question = preprocess_text(line.split('\t')[1])
    vector = np.zeros(100)
    for word in question:
        if word in model_w2v_prod.wv:
            vector += model_w2v_prod.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index_prod.add_item(counter, vector)
    
    counter += 1
    
index_prod.build(10)



True

In [33]:
question = 'юбка детская Orby'
get_answer(question)

'58e3cfe6132ca50e053f5f82  Юбка детская ORBY'

In [34]:
index_prod.save('prod_reply.ann')

True

# Chat


 - Все вопросы из датасета свёрнуты Word2Vec в векторное представление.
 - Построен индекс по вопросам.
 - На запрос в болталку происходит поиск ближайшего вопроса и возвращается ответ на этот вопрос.

## Preprocessing 

## Vectorizing, saving model  

In [35]:
with open('chat_model_l.pkl', 'rb') as file: 
    chat_model = pickle.load(file)

In [36]:
chat_model.wv

## Index 

In [41]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
    for line in f:
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_text(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in chat_model.wv:
                vector += chat_model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
#index.save('sp.ann')

True

# Autotests 

In [42]:
assert(get_answer('Юбка детская ORBY').startswith('58e3cfe6132ca50e053f5f82'))

In [43]:
assert(not get_answer('Где ключи от танка').startswith('5')) 

In [44]:
question = "юбка детская ORby"
get_answer(question)

'58e3cfe6132ca50e053f5f82  Юбка детская ORBY'

In [45]:
question = 'где ключи от танка'
get_answer(question)

'Ща проверим!. \n'

Thank you for attention, it was a real pleasure to share this project to you.